In [50]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from helpers import dataframe
from tensorflow import keras

In [51]:
base_url = "https://raw.githubusercontent.com/italha-d/Stress-Predict-Dataset/main/Raw_data"

# subjects = ["S01", "S02", "S03"]
subjects = [
    "S01", "S02", "S03", "S04", "S05", "S06", "S07", "S08", "S09", "S10",
    # "S11", "S12", "S13", "S14", "S15", "S16", "S17", "S18", "S19", "S20",
    # "S21", "S22", "S23", "S24", "S25", "S26", "S27", "S28", "S29", "S30",
    # "S31", "S32", "S33", "S34", "S35"
]

fetch_objects = dataframe.generate_fetch_objects(base_url, subjects)

In [52]:
acc_df, hr_df, eda_df, temp_df, bvp_df = dataframe.concatenate_dataframe_from_fetch_objects(fetch_objects)

df = dataframe.combine_dataframe(acc_df, hr_df, eda_df, temp_df, bvp_df)

df = dataframe.fill_missing_values(df)

In [53]:
tags_object = dataframe.generate_tag_objects(base_url, subjects)

timestamps = dataframe.fetch_timestamps_from_tag_objects(tags_object)

df = dataframe.label_dataframe_from_timestamps(df, timestamps)

In [13]:
df.to_csv("data.csv", index=False)

### Combined dataset

In [17]:
df.reset_index(inplace=True)
df.set_index(["Subject", "Timestamp"], inplace=True)

In [28]:
data = df.iloc[:, 1:].values
y = df['Label'].values.reshape(-1, 1)

sequence_length = 10
batch_size = 16

dataset = keras.utils.timeseries_dataset_from_array(
    data=data,
    targets=y,
    sequence_length=sequence_length,
    batch_size=batch_size
)

num_train_samples = int(len(dataset) * 0.8)
train_dataset = dataset.take(num_train_samples)
test_dataset = dataset.skip(num_train_samples)

2023-04-20 04:46:27.359848: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-04-20 04:46:27.360349: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-20 04:46:27.360649: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (yoga7): /proc/driver/nvidia/version does not exist
2023-04-20 04:46:27.363646: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [30]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(sequence_length, data.shape[1])),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [31]:
epochs = 10

model.fit(train_dataset, epochs=epochs)

Epoch 1/10
32463/32463 [==============================] - 50s 2ms/step - loss: 0.4196 - accuracy: 0.9904
Epoch 2/10
32463/32463 [==============================] - 54s 2ms/step - loss: 0.3108 - accuracy: 0.9924
Epoch 3/10
32463/32463 [==============================] - 53s 2ms/step - loss: 0.2270 - accuracy: 0.9937
Epoch 4/10
32463/32463 [==============================] - 52s 2ms/step - loss: 0.1497 - accuracy: 0.9952
Epoch 5/10
32463/32463 [==============================] - 54s 2ms/step - loss: 0.1239 - accuracy: 0.9959
Epoch 6/10
 9632/32463 [=======>......................] - ETA: 38s - loss: 0.0947 - accuracy: 0.9955

KeyboardInterrupt: 

### Subject division

In [54]:
new_df = df.set_index(['Subject', 'Timestamp'])

new_df

X     Y     Z     HR       EDA   TEMP     BVP  Label
Subject Timestamp                                                           
1.0     1.644226e+12 -2.0  43.0  43.0  83.00  0.000000  28.39    0.00    0.0
        1.644226e+12 -2.0  44.0  46.0  83.00  0.000000  28.39    0.00    0.0
        1.644226e+12  2.0  44.0  47.0  83.00  0.000000  28.39    0.00    0.0
        1.644226e+12  3.0  43.0  44.0  83.00  0.000000  28.39    0.00    0.0
        1.644226e+12  1.0  43.0  45.0  83.00  0.000000  28.39    0.00    0.0
...                   ...   ...   ...    ...       ...    ...     ...    ...
10.0    1.644847e+12  3.0  -3.0  63.0  76.53  0.306487  26.89 -115.03    0.0
        1.644847e+12  3.0  -3.0  63.0  76.53  0.306487  26.89 -129.06    0.0
        1.644847e+12  3.0  -3.0  63.0  76.53  0.306487  26.89 -144.23    0.0
        1.644847e+12  3.0  -3.0  63.0  76.53  0.306487  26.89 -155.62    0.0
        1.644847e+12  3.0  -3.0  63.0  76.53  0.306487  26.89 -158.14    0.0

[2113911 rows x 8 columns]

In [55]:
from sklearn.model_selection import train_test_split

train_subjects, test_subjects = train_test_split(new_df.index.get_level_values('Subject').unique(), test_size=0.2)

In [56]:
from sklearn.linear_model import LogisticRegression

logistic_regression_model = LogisticRegression()
logistic_regression_model.fit(new_df.loc[train_subjects].drop('Label', axis=1), new_df.loc[train_subjects]['Label'])

logistic_regression_model.score(new_df.loc[test_subjects].drop('Label', axis=1), new_df.loc[test_subjects]['Label'])

0.7703111534864959